In [ ]:
!pip install transformers==4.1.1 sentencepiece==0.1.94
!pip install mosestokenizer==1.1.0
!pip install numpy
!pip install jsonlines

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 5.2 MB/s 
     |████████████████████████████████| 1.1 MB 37.2 MB/s 
     |████████████████████████████████| 2.9 MB 36.7 MB/s 
     |████████████████████████████████| 880 kB 43.4 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=7423e0cec2c3ba59c86959115907643c1457d13a0c2d01e7bd5aafdf176a51d1
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for mosestokenizer: filename=mosestokenizer-1.1.0-py3-none-any.whl size=49118 sha256=1b3f6bc40fd240d47c3c94a5a952d44114bd6617e8eacbcea50f509c84aab200
  Stored in directory: /root/.cache/pip/wheels/a7/31/94/fef279382208e85a65c1a7f5c4d0020115477b0af74f296b57
 

In [ ]:
import json
import numpy
import os
import re
import jsonlines

In [ ]:
from transformers import MarianMTModel, MarianTokenizer


In [ ]:
target_model_name = 'Helsinki-NLP/opus-mt-en-fr'
target_tokenizer = MarianTokenizer.from_pretrained(target_model_name)
target_model = MarianMTModel.from_pretrained(target_model_name)

Downloading:   0%|          | 0.00/778k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/802k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/301M [00:00<?, ?B/s]

Some weights of MarianMTModel were not initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-fr and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def translate(texts, model, tokenizer, language="fr"):
    # Prepare the text data into appropriate format for the model
    template = lambda text: f"{text}" if language == "en" else f">>{language}<< {text}"
    src_texts = [template(text) for text in texts]

    # Tokenize the texts
    encoded = tokenizer.prepare_seq2seq_batch(src_texts, return_tensors='pt')
    
    # Generate translation using model
    translated = model.generate(**encoded)

    # Convert the generated tokens indices back into text
    translated_texts = tokenizer.batch_decode(translated, skip_special_tokens=True)
    
    return translated_texts

In [ ]:
path = '/content/input_texts'
output_path = '/content/output_texts'

In [ ]:
files = os.listdir(path)
files

['mapping_original.jsonl']

In [ ]:
mapped = []
for file_name in files:
  if(file_name != '.ipynb_checkpoints'):
    file_path = os.path.join(path, file_name) # склеивание пути файла 
    with jsonlines.open(file_path, 'r') as reader:
      mapped = mapped + reader.read()

In [ ]:
step = 10

In [ ]:
for start in range(0, len(mapped), step):
  to_translate = [cut['translation']['en'] for cut in mapped[start:start+step]]
  tr_cut = translate(to_translate, target_model, target_tokenizer, language='en')
  j = 0
  for i in range(start, start+step):
    if(i < len(mapped)):
      mapped[i]['translation']['fr'] = tr_cut[j]
    j = j + 1 

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1065: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


In [ ]:
with jsonlines.open('mapping_predictions.jsonl', mode='w') as writer:
        writer.write(mapped)